### 논문용 실험 모듈 구현

1. genus로 사진 읽기.



### ablation study
- species classification vs. hierarchical classification
  - softmax classification on a specific genus test
  - hierarchical classification on a specific genus test
- 





### 데이터셋의 주요 연관정보 출력

In [1]:
from config.path import PATH
from os.path import join
import json

with open(join(PATH["PLANTNET-300K"], "plantnet300K_species_id_2_name.json"), 'r') as file:
    label_to_species = json.load(file) #label => name
with open(join("data/plantnet", "species_to_fine.json"), 'r') as file:
    species_to_fine = json.load(file) #label => name
with open(join("data/plantnet", "fine_to_coarse.json"), 'r') as file:
    fine_to_coarse = json.load(file) #label => name
with open(join("data/plantnet", "genera_to_coarse.json"), 'r') as file:
    genera_to_coarse = json.load(file) #label => name
with open(join("data/plantnet", "genera_to_species.json"), 'r') as file:
    genera_to_species = json.load(file) #label => name

In [2]:
len(genera_to_species)

303

In [3]:
#genera에 속한 species의 종류 계산

species_count = {genera:len(genera_to_species[genera]) for genera in list(genera_to_species.keys())}
sorted_species_count = sorted(species_count.items(), key = lambda item: item[1], reverse = True)

num_to_genera = {}
for genera, num in sorted_species_count:
    if num in num_to_genera.keys():
        num_to_genera[num].append(genera)
    else:
        num_to_genera[num] = [genera]

In [4]:
test_genera = num_to_genera[10] + num_to_genera[11] + num_to_genera[12]
print(test_genera)

['Nymphaea', 'Daphne', 'Smilax', 'Lamium', 'Lavandula', 'Anthurium', 'Dendrobium']


### 학습 및 테스트용 데이터로더 준비

특정 genera의 리스트에 대해 data loader 만들기.

In [7]:
#genera_name = 'Nymphaea'

coarse_id = [genera_to_coarse[genera_name] for genera_name in test_genera]
species_list = []
for genera_name in test_genera:
    species_list+= genera_to_species[genera_name]
fine_id = [species_to_fine[species] for species in species_list]

coarse_to_level1 = {}
fine_to_level2 = {}

for i, c_id in enumerate(coarse_id):
    coarse_to_level1[c_id] = i

for i, f_id in enumerate(fine_id):
    fine_to_level2[f_id] = i

In [8]:
from data.PlantNet import HierarchicalPlantNet, PlantNet
from experiment.plantnet import transforms
split = 'test'
dataset = PlantNet(PATH["PLANTNET-300K"], split=split, shuffle=(split=='train'), transform=transforms[split])
dataset.label_to_class, dataset.class_to_name, dataset.name_to_label = dataset.labels(species_list)
dataset.filelist = dataset.get_filelist()

In [9]:
for genera in test_genera:
    print(f"--------------------{genera}--------------------")
    for species in genera_to_species[genera]:
        print(species, dataset.name_to_label[species])

--------------------Nymphaea--------------------
Nymphaea nouchali Burm. f. 1356309
Nymphaea alba L. 1359669
Nymphaea lotus L. 1362080
Nymphaea candida C.Presl 1393980
Nymphaea nouchali Burm.f. 1409296
Nymphaea ampla (Salisb.) DC. 1409839
Nymphaea mexicana Zucc. 1412831
Nymphaea odorata Aiton 1412833
Nymphaea tetragona Georgi 1412834
Nymphaea rubra Roxb. ex Andrews 1608440
--------------------Daphne--------------------
Daphne gnidium L. 1360588
Daphne laureola L. 1360590
Daphne oleoides Schreb. 1360591
Daphne alpina L. 1391648
Daphne cneorum L. 1391649
Daphne mezereum L. 1391652
Daphne striata Tratt. 1391654
Daphne sericea Vahl 1399157
Daphne odora Thunb. 1420792
Daphne tangutica Maxim. 1420796
--------------------Smilax--------------------
Smilax aspera L. 1363336
Smilax anceps Willd. 1387284
Smilax rotundifolia L. 1398567
Smilax herbacea L. 1413832
Smilax tamnoides L. 1413835
Smilax china L. 1430515
Smilax bona-nox L. 1438352
Smilax glauca Walter 1438354
Smilax laurifolia L. 1438358


In [ ]:
# load dataset 
from data.PlantNet import HierarchicalPlantNet, PlantNet
from config.path import PATH
from experiment.plantnet import transforms
from torch.utils.data import DataLoader
data_loaders = {}

for split in ['train', 'val', 'test']:
    dataset = PlantNet(PATH["PLANTNET-300K"], split=split, shuffle=(split=='train'), transform=transforms[split])
    dataset.label_to_class, dataset.class_to_name, dataset.name_to_label = dataset.labels(species_list)
    dataset.filelist = dataset.get_filelist()
    data_loaders[split] = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

In [37]:
dataset = HierarchicalPlantNet(PATH["PLANTNET-300K"], split="test", shuffle=(split=='train'), transform=transforms[split])
dataset.label_to_class, dataset.class_to_name, dataset.name_to_label = dataset.labels(species_list)
dataset.filelist = dataset.get_filelist()


new_fine_to_coarse = {}
for c, n in dataset.class_to_name.items():
    new_fine_to_coarse[c] = test_genera.index(n.split(" ")[0])
dataset.fine_to_coarse = new_fine_to_coarse


In [40]:
# load dataset 
from data.PlantNet import HierarchicalPlantNet
from config.path import PATH
from experiment.plantnet import transforms
from torch.utils.data import DataLoader
data_loaders = {}


for split in ['train', 'val', 'test']:
    dataset = HierarchicalPlantNet(PATH["PLANTNET-300K"], split=split, shuffle=(split=='train'), transform=transforms[split])
    dataset.label_to_class, dataset.class_to_name, dataset.name_to_label = dataset.labels(species_list)
    dataset.filelist = dataset.get_filelist()
    new_fine_to_coarse = {}
    for c, n in dataset.class_to_name.items():
        new_fine_to_coarse[str(c)] = test_genera.index(n.split(" ")[0])
    dataset.fine_to_coarse = new_fine_to_coarse
    data_loaders[split] = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

내 목적은 다양한 옵션들에서 flat classification보다 더 나은 성능을 나타내는 학습 방법을 찾는 것임.

일단 baseline은...
선택한 7개의 속에 속한 종들에 대해 학습 후 성능을 테스트한다.
['Nymphaea', 'Daphne', 'Smilax', 'Lamium', 'Lavandula', 'Anthurium', 'Dendrobium']

In [ ]:
fine_to_species = dict([(value, key) for key, value in species_to_fine.items()])
class_to_name = {}
for fine_id, level2 in fine_to_level2.items():
    class_to_name[level2] = fine_to_species[fine_id]
print(class_to_name)

In [42]:
from models.ImageClassifier import HierarchicalClassifier
from models.Efficientnet import EfficientB4
from utils.hierarchical_loss import HierarchicalLossNetwork
from torch import nn

baseline_weight_path = "/home/files/experiments/plantnet/baseline/EfficientB4/genera/checkpoints/checkpoint.pt"
baseline = EfficientB4(num_classes=303, loss_fn=nn.CrossEntropyLoss()) #get your model
baseline.load(baseline_weight_path) # load Its the best checkpoint.
baseline.fc = nn.Linear(in_features=1792, out_features=len(species_list), bias=True)
model = HierarchicalClassifier(loss_fn=HierarchicalLossNetwork(new_fine_to_coarse, "cpu"), baseline=baseline, num_classes=[len(test_genera), len(species_list)]).to("cpu")


In [45]:
for name, param in model.named_parameters():
    if 'baseline' in name:
        param.requires_grad = False
    else:
        param.requires_grad = True
    print(name, param.requires_grad)

baseline.patch_embedding.0.weight False
baseline.patch_embedding.1.weight False
baseline.patch_embedding.1.bias False
baseline.patch_embedding.2.0.0.conv_dw.weight False
baseline.patch_embedding.2.0.0.bn1.weight False
baseline.patch_embedding.2.0.0.bn1.bias False
baseline.patch_embedding.2.0.0.se.conv_reduce.weight False
baseline.patch_embedding.2.0.0.se.conv_reduce.bias False
baseline.patch_embedding.2.0.0.se.conv_expand.weight False
baseline.patch_embedding.2.0.0.se.conv_expand.bias False
baseline.patch_embedding.2.0.0.conv_pw.weight False
baseline.patch_embedding.2.0.0.bn2.weight False
baseline.patch_embedding.2.0.0.bn2.bias False
baseline.patch_embedding.2.0.1.conv_dw.weight False
baseline.patch_embedding.2.0.1.bn1.weight False
baseline.patch_embedding.2.0.1.bn1.bias False
baseline.patch_embedding.2.0.1.se.conv_reduce.weight False
baseline.patch_embedding.2.0.1.se.conv_reduce.bias False
baseline.patch_embedding.2.0.1.se.conv_expand.weight False
baseline.patch_embedding.2.0.1.se.con

In [ ]:
baseline

In [ ]:
baseline

In [ ]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0"
baseline_weight_path = "/home/files/experiments/plantnet/baseline/EfficientB4/genera/checkpoints/checkpoint.pt"
loaded_dict = torch.load(baseline_weight_path,map_location="cuda:0")